In [1]:
import numpy as np
from tensorflow import keras

In [2]:
from keras import Input, layers
from keras.models import Sequential, Model

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

Using TensorFlow backend.


In [3]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)
model = Model(input_tensor, output_tensor)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


## 7.1.2 Multi-input models

* 関数型の API を使えば，複数の入力があるニューラルネットワークを構築することができる．
* `keras.layers.add` や `keras.layers.concatenate` などで結合する

In [5]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

# text を 64 次元のベクトルに埋め込むレイヤ
embedded_text = layers.Embedding(
    input_dim=text_vocabulary_size,
    output_dim=64)(text_input)

# 埋め込まれたベクトルのコレクション (BachSize, 64) を，ひとつの 32 次元ベクトルに変換
encoded_text = layers.LSTM(32)(embedded_text)

# question についても同様に
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# encode された２つのベクトルを結合する
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# 全結合層で最終出力
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# テンソル answer の計算グラフでモデルを構築
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
             metrics=['acc'])

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS

In [7]:
# データを作る

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

In [8]:
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 7s 7ms/step - loss: 6.2149 - acc: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2004 - acc: 0.0490
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1806 - acc: 0.0510
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.0979 - acc: 0.0080
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.0245 - acc: 0.0070
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.9543 - acc: 0.0070
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.8626 - acc: 0.0120
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.7611 - acc: 0.0110
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6745 - acc: 0.0170
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6133 - acc: 0.0170


## 7.1.3 Multi-output models

In [9]:
vocabulary_size = 50000
n_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# この x : shape=(?, 128) から各属性を予測する
age_predection = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(n_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

# 単純に outputs=[...] とすれば良い
model = Model(posts_input, [age_predection, income_prediction, gender_prediction])

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [10]:
# ロス関数の指定はディクショナリかリストで渡す

loss_list = ['mse', 'categorical_crossentropy', 'binary_crossentropy']
loss_list_dict = {name:value for name,value in zip(model.output_names, loss_list)}

loss_weights_list = [0.25, 1, 10]
loss_weights_dict = {name:value for name,value in zip(model.output_names, loss_weights_list)}

model.compile(optimizer='rmsprop', loss=loss_list, loss_weights=loss_weights_list)

## 7.1.4 Directed acyclic graphs of layers

### Inception

* GoogLeNet で使用された，レイヤーを組み合わせたモジュール (ネットワーク アーキテクチャ)
* 元は network-in-network
* 空間的な特徴（W, H）と，チャンネル方向 (K) の特徴を別々に学習することで，効率よく学習するのが狙い
* この本の作者である Francois Chollet らによって，*Xception* という名前の改良版が開発された

**1x1 の CNN**

* 入力層が $W\times H\times K$ なら，出力層は $W\times H\times 1$
* パラメタ数は $1\times 1\times K$
* チャネル方向での結合を行うので，チャネルの削減に使われる（チャネル削減した後に Conv したり）

### Residual connections

* 2015 年に提案されたネットワークのアーキテクチャで，Xception モデルを使用している
* Microsoft の He らによって提案された
* ILSVRC の ImageNet 2015 で優勝したモデル
* 大規模なネットワークが抱える問題に挑戦
    * 勾配消失問題
    * 表現的ボトルネック

#### どんな工夫？

* 前のレイヤーの出力を，レイヤーをいくつか飛ばしたあとの入力に結合する（要素和ではない）

```py
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])
```

#### 表現的ボトルネック (representational bottleneck)

`Sequential` モデルにおいて，それぞれのレイヤーはそれより前にある複数層の出力にもとづいて計算される．
もし，どこかに小さい層（e.g. 非常に低次元な特徴しか持たない層）があれば，モデルは「この層の活性値にどれだけ情報をつめこめるか？」という問題と戦わなければならない．
これは非常に強い制約であり，**representational bottleneck** として知られている．

# 7.2 Keras callback と TensorBoard を使ったモデルの観察

バリデーションセットのロスが改善しなくなれば学習を途中でやめたいときがある．
これは毎エポックの決まったタイミングで実行されるコールバック関数を定義することで実現できる．
コールバック関数を使えば，他にも

* *Model Checkpointing*：現在のモデルを学習途中で保存する
* *Early Stopping*：validation loss が改善しなくなった時点で学習を中断する
* *Dynamicaly Adjusting the value of certain parameters during training*: 学習率やオプティマイザの動的計画
* *Logging training and validation metrics during training, or visualizing the representations learned by the models as there updated*: プログレスパーなどで学習の様子を表示したり，ログを残す

などが実行できる．

In [5]:
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

17465344/17464789 [==============================] - 5s 0us/step


In [44]:
x_train

(25000, 500)

In [72]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf


model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D()) # global 系プーリング層は，paddingなしのshape=(W,H)でチャネル数1のプーリングと同じ
model.add(Dense(1))

model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

callbacks = [
    keras.callbacks.TensorBoard(histogram_freq=1)
]
history = model.fit(x_train, y_train,
                   epochs = 20,
                   batch_size=128,
                   validation_split=0.2,
                   callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________
Trai

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("embed_input:0", shape=(?, 500), dtype=float32) is not an element of this graph.

In [63]:
keras.callbacks.TensorBoard?

Init signature: keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False)
Docstring:     
Tensorboard basic visualizations.

This callback writes a log for TensorBoard, which allows
you to visualize dynamic graphs of your training and test
metrics, as well as activation histograms for the different
layers in your model.

TensorBoard is a visualization tool provided with TensorFlow.

If you have installed TensorFlow with pip, you should be able
to launch TensorBoard from the command line:

```sh
tensorboard --logdir=/full_path_to_your_logs
```

You can find more information about TensorBoard
[here](https://www.tensorflow.org/get_started/summaries_and_tensorboard).

Arguments:
    log_dir: the path of the directory where to save the log
        files to be parsed by TensorBoard.
    histogram_freq: frequency (in epochs) at which to compute activation
        and weight histograms for the layers of the model. If